In [2]:
import cv2
import numpy as np
import os
import pandas as pd
import datetime

In [23]:
def process_time_string(time_string):
    time_string = str(time_string)
    if time_string == 'nan':
        return None
    if len(time_string) > 14:
        millis = float(time_string[14:])
    else:
        millis = 0
    secs = int(time_string[12:14])
    mins = int(time_string[10:12])
    hours = int(time_string[8:10])
    day = int(time_string[6:8])
    month = int(time_string[4:6])
    year = int(time_string[0:4])
    # divide millis until there is nothing to the right of the decimal point
    while millis > 1:
        millis /= 10
    return datetime.datetime(year, month, day, hours, mins, secs, int(millis * 1000))

def scrub_video_seconds(vid_capture, seconds):
    fps = vid_capture.get(cv2.CAP_PROP_FPS)
    frame_count = int(vid_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    current_pos_seconds = frame_count / fps
    time_to_skip = current_pos_seconds - seconds
    for i in range(int(time_to_skip * fps)):
        vid_capture.read()
    return vid_capture

In [48]:
# get list of video names from the folder of processed CSVs
csv_times = [ process_time_string(name.split('_')[0]) for name in os.listdir('successful_trials_by_day')]
csv_paths = [name for name in os.listdir('successful_trials_by_day')]
# iterate through each csv and find the closest video name
video_times = [process_time_string(name.split('_')[0]) for name in os.listdir('Data_with mouse ID/videos/')]
video_paths = [name for name in os.listdir('Data_with mouse ID/videos/')]
csv_vid_match = dict()
for csv_time in csv_times:
    closest_time = video_times[0]
    closest_path = video_paths[0]
    for i, video_time in enumerate(video_times):
        if abs((csv_time - video_time).total_seconds()) <  abs((csv_time - closest_time).total_seconds()):
            closest_time = video_time
            closest_path = video_paths[i]
    total_delta = (csv_time - closest_time).total_seconds()
    if total_delta < 10:
        csv_vid_match[csv_paths[csv_times.index(csv_time)]] = {
            'video_name': closest_path,
            'delta_seconds': total_delta
        }

   
    
# iterate throught each video and load the corresponding video and csv
for csv_name in csv_vid_match.keys():
    video = cv2.VideoCapture('Data_with mouse ID/videos/' + csv_vid_match[csv_name]['video_name'])
    df = pd.read_csv('successful_trials_by_day/' + csv_name)
    if len(df['successful_trials']) < 10:
        continue
    # iterate through df and for each successfull trial get the start time in seconds
    for index, row in df.iterrows():
        if row['successful_trials']:
            start_time = row['time_since_start']
            video_time_offset = csv_vid_match[csv_name]['delta_seconds']
            # process start_time with datetime
            start_time = start_time.split(' ')[2].split(':')
            start_time = float(start_time[0])*3600 + float(start_time[1])*60 + float(start_time[2]) + csv_vid_match[csv_name]['delta_seconds'] + video_time_offset
            end_time = start_time + 15
            # set the video to the start time
            fps = video.get(cv2.CAP_PROP_FPS)
            current_time = float(video.get(cv2.CAP_PROP_POS_FRAMES)) / fps
            time_offset = start_time - current_time
            for i in range(int(time_offset * video.get(cv2.CAP_PROP_FPS))):
                video.read()

            # find the frames where mouse R and L break the beam
            R_time_offset = row['R']
            L_time_offset = row['L']
            R_break_frame = R_time_offset * fps + start_time * fps
            L_break_frame = L_time_offset * fps + start_time * fps
            
            # find the mice names
            name1 = row['names'][:2]
            name2 = row['names'][2:]
            # display the video until the end time
            while True:
                success, image = video.read()
                if success == False or float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps > end_time:
                    break
                # add a countdown to the video 
                cv2.putText(image, str(int(end_time - float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                # display the mice names
                cv2.putText(image, name1, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, name2, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.imshow('frame', image)
                # display second frame cropped from (247, 285) to (383, 432)
                cv2.imshow('frame2', image[275:432, 222:383])
                cv2.waitKey(12)

    

In [ ]:
for cvpath in video_names:
    print(cvpath in os.listdir('Data_with mouse ID/videos/'))
    video = cv2.VideoCapture('Data_with mouse ID/videos/' + cvpath)
    # show the whole video in a window
    while True:
        success, image = video.read()
        cv2.imshow('frame', image)
        cv2.waitKey(12)
        if success == False:
            break
    
    

False


[ERROR:0@309.913] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'




error: OpenCV(4.8.0) /io/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
